# [Routing](markdown/routing-overview.md)

In [1]:
from dotenv import load_dotenv
import os

## [Part 10: Logical routing](markdown/logical-routing.md)

In [7]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )


# LLM with function call 
llm = ChatOllama(model="qwen2.5:32b-instruct-q4_K_M", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, choose one of the following datasources:
- 'python_docs' for Python-related questions
- 'js_docs' for JavaScript-related questions
- 'golang_docs' for Go-related questions.

Only respond with one of the above options."""


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [8]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

result = router.invoke({"question": question})

In [9]:
print(result)

datasource='python_docs'
